We are going to create agents that can deep search

In [37]:
# Necessary imports
from agents import Agent, Runner, trace, WebSearchTool, function_tool
from dotenv import load_dotenv
from agents.model_settings import ModelSettings
from openai import OpenAI
from pydantic import BaseModel, Field
import os
import asyncio
from typing import Dict
import sendgrid
from sendgrid.helpers.mail import Mail
from IPython.display import Markdown, display

In [38]:
load_dotenv(override=True)

True

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.

### Important note - API charge of WebSearchTool

This is costing me 2.5 cents per call for OpenAI WebSearchTool. That can add up to $2-$3 for the next 2 labs. We'll use free and low cost Search tools with other platforms, so feel free to skip running this if the cost is a concern. Also student Christian W. pointed out that OpenAI can sometimes charge for multiple searches for a single call, so it could sometimes cost more than 2.5 cents per call.

Costs are here: https://platform.openai.com/docs/pricing#web-search

In [39]:
# Create a search agent
INSTRUCTIONS = """You are a research assistant. Given a search term, you search the web for that term and produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 
words. Capture the main points. Write succintly, no need to have complete sentences or good grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the essence and ignore any fluff. Do not include any additional commentary other than the summary itself.
"""
search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")], 
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [40]:
message = "How to start a business in India that is profitable?"
with trace("Research"):
    result = await Runner.run(search_agent, message)
    
display(Markdown(result.final_output))

Starting a profitable business in India involves identifying high-demand sectors, understanding market dynamics, and leveraging available resources. Key profitable business ideas include:

- **Cloud Kitchens**: Operate delivery-only kitchens with low overhead costs, catering to the growing demand for online food delivery. ([flexiloans.com](https://flexiloans.com/blog/top-10-most-profitable-business-ideas-in-india-for-2025/?utm_source=openai))

- **Spice Processing and Packaging**: Capitalize on India's spice production by processing and packaging spices for domestic and export markets. ([finline.in](https://www.finline.in/resource/small-business-ideas/?utm_source=openai))

- **Event Management Services**: Organize weddings, corporate events, and parties, tapping into India's celebration culture. ([matchscorecard.org](https://matchscorecard.org/which-business-is-most-profitable-in-india/?utm_source=openai))

- **Organic Farming**: Grow chemical-free produce to meet the rising demand for organic products. ([bdtask.com](https://www.bdtask.com/blog/most-profitable-business-in-india?utm_source=openai))

- **Health & Wellness Business**: Offer fitness training, yoga classes, and wellness products, leveraging the growing health consciousness. ([startupstag.com](https://startupstag.com/blog/profitable-startup-ideas-in-india/?utm_source=openai))

- **Mobile App Development**: Develop apps for businesses, capitalizing on the smartphone boom and digitalization. ([matchscorecard.org](https://matchscorecard.org/which-business-is-most-profitable-in-india/?utm_source=openai))

- **Fish Farming (Aquaculture)**: Engage in fish farming, especially in coastal areas, to meet the increasing seafood demand. ([finline.in](https://www.finline.in/resource/top-successful-small-business-ideas/?utm_source=openai))

- **Customized Gifts and Event Decor**: Provide personalized gifting and décor services, catering to various celebrations. ([flexiloans.com](https://flexiloans.com/blog/top-10-most-profitable-business-ideas-in-india-for-2025/?utm_source=openai))

- **Cold Chain & Temperature-Controlled Warehousing**: Offer storage solutions for perishable goods, supporting sectors like food delivery and pharmaceuticals. ([thelogicstick.com](https://thelogicstick.com/top-11-most-profitable-business-ideas-in-india/?utm_source=openai))

- **Premium D2C Skincare & Beauty Brand**: Launch a direct-to-consumer skincare brand, tapping into the growing beauty market. ([thelogicstick.com](https://thelogicstick.com/top-11-most-profitable-business-ideas-in-india/?utm_source=openai))

To establish a profitable business in India, conduct thorough market research, develop a solid business plan, and ensure compliance with local regulations. Leveraging digital platforms for marketing and sales can also enhance reach and profitability.

For a comprehensive overview of profitable business ideas in India, you might find this video helpful:

[Top 30 Business Ideas in India](https://www.youtube.com/watch?v=WUtgAfjBGnk&utm_source=openai)
 

In [41]:
# We now use structured output to get a better result for planner_agent
HOW_MANY_SEARCHES = 3
INSTRUCTIONS = f"""You are a research assistant. Given a query you need to come with {HOW_MANY_SEARCHES} search terms. Be careful in providing the search term as these search terms provided by you will be searched on web to find the answer
for the query. Your goal should be providing the effective search terms that when it gets searched the search results should help answring the query."""

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search term is helpful in answering the query")
    search_term: str = Field(description="The search term to use for the web search")

class WebSearchPlan(BaseModel):
    search_terms:list[WebSearchItem] = Field(description="A list of all the search terms for web search to better answer the question")

web_search_planner_agent = Agent(name="Web Search Planner",
instructions=INSTRUCTIONS,
model="gpt-4o-mini",
output_type=WebSearchPlan)



In [42]:
message = "I need some ideas to start a profitable business in 2026"
with trace("WebSearchPlanner"):
    result = await Runner.run(web_search_planner_agent, message)
print(result.final_output)

search_terms=[WebSearchItem(reason='To explore emerging trends and opportunities in the business landscape for 2026.', search_term='profitable business ideas 2026 trends'), WebSearchItem(reason='To identify industries expected to grow significantly by 2026 and viable business options within them.', search_term='future profitable industries 2026'), WebSearchItem(reason='To find insights from experts and entrepreneurs on new business opportunities and innovative ideas for 2026.', search_term='entrepreneurship opportunities 2026')]


In [43]:
# Create a report writer agent
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)

class ReportData(BaseModel):
    short_summary:str = Field(description="A short 2-3 sentence summary for the finding")
    markdown_report:str = Field(description="The Final Report")
    follow_up_questions:list[str] = Field(description="suggested topics to research further")

report_writer_agent = Agent(name="Report Writer",
instructions=INSTRUCTIONS,
model="gpt-4o-mini",
output_type=ReportData)

In [44]:
# Crate a tool to send emails using SendGrid
from sendgrid import html_content


@function_tool
def send_html_email(subject:str, html_body:str)->dict[str,str]:
    """ Send out an email with provided subject and HTML Body"""
    sg = sendgrid.SendGridAPIClient(os.getenv("SENDGRID_API_KEY"))
    message = Mail(from_email="sumanprakash.nitp@gmail.com",
    to_emails="bittu1suman2@gmail.com",
    subject=subject,
    html_content=html_body)
    sg.send(message)
    return {"Status":"Success"}

In [45]:
# Create an email agent who can use send_html_tool to send a well formatter HTML with a detail report
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(name="Email Agent",
instructions=INSTRUCTIONS,
model="gpt-4o-mini",
tools=[send_html_email])

In [46]:
# Now create function to cohesively search the search terms, generate report, and sending email
async def plan_searches(query:str):
    """Use the web_search_planner_agent to plan the search terms that need to be searched on web"""
    print("Planning Search Terms...")
    result = await Runner.run(web_search_planner_agent, f"Query:{query}")
    print(f"Will perform {len(result.final_output.search_terms)} searches")
    return result.final_output


async def search(search_item:WebSearchItem):
    """Use the search_agent to do a websearch for the given search_item"""
    input = f"Search term:{search_item.search_term}\nReason for search:{search_item.reason}"
    result= await Runner.run(search_agent,input)
    return result.final_output

async def perform_searches(search_plan:WebSearchPlan):
    """Execute search() for each item in search_terms"""
    print("Searching...")
    tasks = [asyncio.create_task(search(search_term)) for search_term in search_plan.search_terms]
    result = asyncio.gather(*tasks)
    print("Finished Searching...")
    return result

async def write_report(query:str, search_results:list[str]):
    """Use the report_writer_agent to write a report based on the search results."""
    print("Thinking about report....")
    input=f"Query:{query}\nSummarized Search Result:{search_results}"
    result= await Runner.run(report_writer_agent, input)
    print("Report Generated...")
    return result.final_output

async def send_email(report:ReportData):
    """use email_agent to send the given report"""
    print("Writing Email...")
    result = await Runner.run(email_agent,report.markdown_report)
    print("Email sent.")
    return report



In [47]:
query= "Get me a lucrative business idea to start in 2026"
with trace("Research"):
    print("Starting research...")
    search_terms = await plan_searches(query)
    search_results = await perform_searches(search_terms)
    report = await write_report(query, search_results)
    await send_email(report)
    print("Research accomplished! Please check traces.")

Starting research...
Planning Search Terms...
Will perform 3 searches
Searching...
Finished Searching...
Thinking about report....
Report Generated...
Writing Email...
Email sent.
Research accomplished! Please check traces.
